# Big Data Content Analytics - AUEB

## Callbacks that reduce training time using Keras

* Lab Assistant: George Perakis
* Email: gperakis[at]aeub.gr 

### Imports

In [1]:
import numpy as np
import pandas as pd

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

from tensorflow.keras.optimizers import SGD

# used to create mock-up data
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [2]:
### Create Mock-up dataset

In [3]:
n_feats = 20

X, y = make_classification(n_samples=100_000,
                           n_features=20,
                           n_informative=3,
                           n_redundant=0,
                           n_classes=2,
                           n_clusters_per_class=2)

pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.379084,-0.937865,-0.297474,0.117213,0.239456,0.894445,2.547377,1.137367,-0.224268,0.559724,-0.182770,0.487008,-0.435420,0.850456,-1.090675,-1.178499,0.933990,-0.194062,-1.012124,-0.299009
1,0.274767,0.931978,0.190487,-0.592741,0.287354,-0.586296,0.109829,-0.911711,0.617918,-2.540849,0.290141,-2.652183,0.213764,-1.621193,-1.590486,0.108250,-0.151673,-3.966430,-2.041659,1.377930
2,-0.664970,0.866321,-0.882064,0.052033,1.323550,-0.286776,-0.463751,-0.697133,-1.573636,1.031752,0.279371,-0.963318,1.762592,0.170219,0.224306,1.826357,-0.766580,-0.949980,-1.103628,0.329588
3,0.084782,0.582900,0.875120,1.645179,0.830084,0.394436,-0.280523,-0.626699,-1.396592,0.638423,1.953321,0.021131,-0.352626,0.241072,-0.226534,1.534620,-0.360904,-1.576495,-0.373071,-0.255314
4,-2.141583,1.279575,1.007061,-1.017441,-0.884840,1.110146,0.367618,1.389828,-1.398979,-0.921820,0.010304,0.976645,-0.684824,-1.091520,-0.522616,1.453910,-0.914803,-0.186849,-0.449709,0.493157


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [5]:
X_train.shape

(75000, 20)

In [6]:
X_train.shape[1]

20

### Build Sequential Model

In [7]:
# instantiate model
model = Sequential()

# we can think of this chunk as the input layer
model.add(Dense(128, input_dim=X_train.shape[1]))

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# we can think of this chunk as the hidden layer    
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# we can think of this chunk as the output layer
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

# setting up the optimization of our weights 
sgd = SGD(lr=0.1,
          decay=1e-6,
          momentum=0.9,
          nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['acc'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2688      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                2

### Adding Callbacks

In [ ]:
monitor = 'val_loss'

model_fname = 'model.h5'

callbacks = [
        
#     TensorBoard(log_dir=TENSORBOARD_LOGS_DIR,
#                 histogram_freq=0,
#                 embeddings_freq=0,
#                 write_graph=True,
#                 write_images=False),
    
    # Stop training when a monitored quantity has stopped improving.
    EarlyStopping(monitor=monitor,
                  patience=5,
                  verbose=1,
                  restore_best_weights=True),
    
    # Save the model after every epoch of the monitor quantity improves
    ModelCheckpoint(filepath=model_fname,
                    monitor=monitor,
                    save_best_only=True,
                    save_weights_only=False,
                    verbose=1),
    
    # Reduce learning rate when a metric has stopped improving
    ReduceLROnPlateau(monitor=monitor,
                      factor=0.1,
                      patience=3,
                      verbose=1)

]

### Model Fit

In [ ]:
# fitting the model on the data
history = model.fit(X_train,
                    y_train,
                    epochs=500,
                    batch_size=16,
                    validation_split=0.2, 
                    verbose = 1,
                    callbacks= callbacks
                   )

In [ ]:
# from tensorflow.python.keras.models import load_model

# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# # returns a compiled model
# # identical to the previous one
# model = load_model('my_model.h5')

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

def plot_keras_history(history):
    """
    
    :param history: 
    :return: 
    """
    # the history object gives the metrics keys. 
    # we will store the metrics keys that are from the training sesion.
    metrics_names = [key for key in history.history.keys() if not key.startswith('val_')]

    for i, metric in enumerate(metrics_names):
        
        # getting the training values
        metric_train_values = history.history.get(metric, [])
        
        # getting the validation values
        metric_val_values = history.history.get("val_{}".format(metric), [])

        # As loss always exists as a metric we use it to find the 
        epochs = range(1, len(metric_train_values) + 1)
        
        # leaving extra spaces to allign with the validation text
        training_text = "   Training {}: {:.5f}".format(metric,
                                                        metric_train_values[-1])

        # metric
        plt.figure(i, figsize=(12, 6))

        plt.plot(epochs,
                 metric_train_values,
                 'b',
                 label=training_text)
        
        # if we validation metric exists, then plot that as well
        if metric_val_values:
            validation_text = "Validation {}: {:.5f}".format(metric,
                                                             metric_val_values[-1])

            plt.plot(epochs,
                     metric_val_values,
                     'g',
                     label=validation_text)
        
        # add title, xlabel, ylabe, and legend
        plt.title('Model Metric: {}'.format(metric))
        plt.xlabel('Epochs')
        plt.ylabel(metric.title())
        plt.legend()

    plt.show()

In [ ]:
plot_keras_history(history)

#### Extra info

In [ ]:
# Extra read
# https://en.wikipedia.org/wiki/Simulated_annealing
# You may create a CLR (cyclic learing rate) callback

# https://github.com/bckenstler/CLR